In [2]:
import pickle
import json
import re
from urllib.request import urlopen
from bs4 import BeautifulSoup
import signal
from urllib.parse import urlparse
import uuid

In [20]:
all_needed_comments = pickle.load(open('../data_2017-09/all_needed_comments.pkl', 'rb'))
url_file_map = pickle.load(open('../data_2017-09/url_file_map.pkl', 'rb'))
valid_urls = pickle.load(open('../data_2017-09/valid_urls.pkl', 'rb'))

In [15]:
print(valid_urls[300])

{'urls': ['https://en.wikipedia.org/wiki/Libertarian_socialism'], 'id': 'dmehp99', 'chain': ['dmehp99']}


In [21]:
tmp = {}
for line in valid_urls:
    if len(valid_urls[line]['chain']) < 2:
        continue
    if valid_urls[line]['chain'] == ['dmehrxc', 'dmehs40']:
        print(line, valid_urls[line])
            
        #if url not in tmp:
        #    tmp[url] = {'filename': uuid.uuid4().hex, 'chains': [valid_urls[line]['chain']]}
        #else:
        #    # takes care of case where one url has many comment chains
        #    tmp[url]['chains'].append(valid_urls[line]['chain'])

3968 {'urls': ['https://en.wikipedia.org/wiki/JAB_Holding_Company'], 'id': 'dmehs40', 'chain': ['dmehrxc', 'dmehs40']}
80138 {'urls': ['https://en.wikipedia.org/wiki/JAB_Holding_Company'], 'id': 'dmehs40', 'chain': ['dmehrxc', 'dmehs40']}
156301 {'urls': ['https://en.wikipedia.org/wiki/JAB_Holding_Company'], 'id': 'dmehs40', 'chain': ['dmehrxc', 'dmehs40']}


In [19]:
with open('../data_2017-09/RC_2017-09', 'r') as f:
        for i,line in enumerate(f):
            if i in [3968, 80138, 156301]:
                comment = json.loads(line)
                print(comment)
            if i > 156301: break

{'author': 'HelperBot_', 'author_flair_css_class': None, 'author_flair_text': None, 'body': 'Non-Mobile link: https://en.wikipedia.org/wiki/JAB_Holding_Company\n***\n^HelperBot ^v1.1 ^/r/HelperBot_ ^I ^am ^a ^bot. ^Please ^message ^/u/swim1929 ^with ^any ^feedback ^and/or ^hate. ^Counter: ^107142', 'can_gild': True, 'collapsed': False, 'collapsed_reason': None, 'controversiality': 0, 'created_utc': 1504224108, 'distinguished': None, 'edited': False, 'gilded': 0, 'id': 'dmehs40', 'is_submitter': False, 'link_id': 't3_6vyqov', 'parent_id': 't1_dmehrxc', 'retrieved_on': 1504556683, 'score': 1, 'stickied': False, 'subreddit': 'minnesota', 'subreddit_id': 't5_2qhxs'}
{'author': 'HelperBot_', 'author_flair_css_class': None, 'author_flair_text': None, 'body': 'Non-Mobile link: https://en.wikipedia.org/wiki/JAB_Holding_Company\n***\n^HelperBot ^v1.1 ^/r/HelperBot_ ^I ^am ^a ^bot. ^Please ^message ^/u/swim1929 ^with ^any ^feedback ^and/or ^hate. ^Counter: ^107142', 'can_gild': True, 'collapsed'

In [4]:
for url in url_file_map:
    if 'status' not in url_file_map[url] or url_file_map[url]['status'] == "failure": continue
    print(url_file_map[url])

{'filename': 'b97b6dd79e12453f82ccc9d37c25e3c1', 'chains': [['dmehrxc', 'dmehs40'], ['dmehrxc', 'dmehs40'], ['dmehrxc', 'dmehs40']], 'status': 'success', 'split': 'train'}
{'filename': 'be93820febe34f57bf438fa041f0ebe5', 'chains': [['dmehz3t', 'dmehza5'], ['dmehz3t', 'dmehza5'], ['dmehz3t', 'dmehza5'], ['dmtd7dy', 'dmu8fj5', 'dmucrax'], ['dmywehm', 'dmz36yy'], ['dn3oifl', 'dn43n9m', 'dn4oi9w', 'dn4pk6l', 'dn4q8gi']], 'status': 'success', 'split': 'train'}
{'filename': '6a88c0812db44e99ab64632eb19f8c4a', 'chains': [['dmehvny', 'dmei1nx'], ['dmehvny', 'dmei1nx'], ['dmehvny', 'dmei1nx'], ['dmehvny', 'dmf2f2k']], 'status': 'success', 'split': 'train'}
{'filename': 'b8b9b6b38fcc495fa22a3c99494c8b59', 'chains': [['dmehtr4', 'dmei28c'], ['dmehtr4', 'dmei28c'], ['dmehtr4', 'dmei28c'], ['dmlcnr6', 'dmldwiv', 'dmljki6', 'dmlktlo', 'dmlux93', 'dmm08c0', 'dmmdyi0', 'dmmrj76']], 'status': 'success', 'split': 'train'}
{'filename': 'e7f175aec51341deb17c63c50e7f1ba2', 'chains': [['dmehshh', 'dmei2vs']

In [22]:
x = '''2767	y lowest IQ's. <C> &gt;Actually they do

No, not to any significant degree. 

The average in America is about 100, give or take 5 points.  <C> Five points is huge, and if you look at the differences between the major ethnic groups with the highest and lowest averages it's 20 points.  Furthermore, it's likely that there are also smaller ethnic groups with very low average IQ, for whom, the difference to the highest major group is 37 points.  There are also smaller groups with IQ averages of 112 and 115 respectively, which would make the IQ difference between the groups with highest and lowest average IQ's 47 points. <C> &gt;Five points is huge

Five points is a bad night, I know from experience. 

&gt;the difference

Exists on a global scale. Not in America.  <C> No, they '''

print([x])

["2767\ty lowest IQ's. <C> &gt;Actually they do\n\nNo, not to any significant degree. \n\nThe average in America is about 100, give or take 5 points.  <C> Five points is huge, and if you look at the differences between the major ethnic groups with the highest and lowest averages it's 20 points.  Furthermore, it's likely that there are also smaller ethnic groups with very low average IQ, for whom, the difference to the highest major group is 37 points.  There are also smaller groups with IQ averages of 112 and 115 respectively, which would make the IQ difference between the groups with highest and lowest average IQ's 47 points. <C> &gt;Five points is huge\n\nFive points is a bad night, I know from experience. \n\n&gt;the difference\n\nExists on a global scale. Not in America.  <C> No, they "]
